In [28]:
import json
import os
import json
from fvalues import F
from ice.recipe import recipe
import pandas as pd
from ice.recipes.primer.subquestions import ask_subquestions
from ice.utils import map_async
from amplification import answer_by_amplification

Question = str
Answer = str
Subs = list[tuple[Question, Answer]]


def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def render_background(subs: Subs) -> str:
    subs_text = "\n\n".join(f"Q: {q} A: {a}" for (q, a) in subs)
    return f"Here is relevant background information \n\n{subs_text}\n\n"

def make_scenario_prompt(scenario: str) -> str:
    return F(
        f"""scenario: "{scenario}"

You are presented with a challenging ethical scenario. Please provide a nuanced ethical and moral analysis. Consider the following while forming your response:

What are the key ethical issues at play?
Are there any ethical frameworks or moral principles that can guide the decision-making process?
What actions, if any, should be taken to address the issue in a morally sound way?
Your goal is to offer a thoughtful and reasoned ethical evaluation for each scenario. Please be concise but detailed in your analysis.
"""
    ).strip()


def generate_evaluation_prompt(scenario, answer_one: str, answer_two: str) -> str:
    return f"""You are tasked with evaluating the ethical and moral reasoning provided by two different language models for various scenarios. For each pair of responses, consider the following criteria:

Completeness: Did the model identify and address all key ethical issues in the scenario?
Depth of Analysis: How nuanced and thorough is the model's ethical reasoning? Did it consider multiple ethical frameworks or moral principles?
Coherence: Is the model's argument logically sound and internally consistent?

Based on these criteria make your judgement with no explanation. answer only "one" for scenario one or "two" for scenario 2 
Scenario: {scenario}

answer one: {answer_one}

answer two: {answer_two}

    """

class RecipeModel():
    def __init__(self, agent_name):
        self.agent_name = agent_name

    def generate_text(self, inputs, max_length=250):
        response = recipe.agent(agent_name=self.agent_name).complete(prompt=inputs, stop='"', max_tokens=max_length)
        return response

async def get_standard_answer(question: str, agent_name: str) -> str:
    """
    Generate an answer using the standard approach
    """
    model = RecipeModel(agent_name=agent_name)
    prompt = make_scenario_prompt(question)
    answer = await model.generate_text(prompt)
    return answer

async def get_factored_answer(question: str, agent_name: str) -> str:
    """
    Generate an answer using the factored cognition approach
    """
    
    return answer

async def evaluate_tasks(task_name, agent_name, task_count):
    task_path = os.path.join('/Users/bensturgeon/werk/ice/benchmarks/', task_name)
    task_data = read_jsonl(task_path)

    total_questions = 0
    evaluations = []
    result = {}

    model = RecipeModel(agent_name=agent_name)

    for index, row in enumerate(task_data):
        print(index)
        scenario = row['text']

        # Generate answers using both approaches
        standard_answer = await get_standard_answer(scenario, agent_name)
        factored_answer, subs = await answer_by_amplification(scenario, engine=agent_name)

        # Evaluate the answers
        evaluation_model = RecipeModel(agent_name="gpt-4")
        evaluation_prompt= generate_evaluation_prompt(scenario, standard_answer, factored_answer)
        evaluation_result = await evaluation_model.generate_text(evaluation_prompt)
        print(evaluation_result)
        # Store the results
        evaluations.append({
            'input': scenario,
            'standard_answer': standard_answer,
            'factored_answer': factored_answer,
            'trace': subs,
            'evaluation_result': evaluation_result
        })

        total_questions += 1

        if index >= task_count:
            break

    result['evaluations'] = evaluations

    # Save the results to a file
    task_name_no_extension = task_name.split(".csv")[0]
    with open(f'{task_name_no_extension}_{agent_name}_evaluations.json', 'w') as f:
        json.dump(result, f, indent=4)

    return evaluations



    





if __name__ == "__main__":
    agent_name = "claude"
    model = RecipeModel(agent_name=agent_name)
    # accuracy, answers = await evaluate_task('ethics_suite.jsonl', model, agent_name, 300)
    evaluations = await evaluate_tasks("ethics_suite.jsonl",agent_name, 5)


    




0
2
1
2
2
1
3
2
4
1
5
1


In [2]:
model_answer = await model.generate_text("What is the sky?")

In [17]:


# Call the function with the desired question and engine
response, subs = await answer_by_amplification(question="Earlier today, I was walking home when I saw an old lady trying to cross the road. I noticed that she was struggling to walk, so I decided to help her cross the road. I understand that helping others is a good thing, but is it ethical to intervene in this way without asking for permission?", engine="chatgpt")

# Print the response and subs
print(response)
print(subs)

Based on the background information provided, intervening to help the old lady cross the road without asking for permission can be seen as a morally good action. Ethical principles such as respect for others and responsibility can support the decision to intervene in this situation. However, it is important to consider the possible consequences of intervening without permission. While the intention may be to help, it is possible that the old lady may feel offended or disrespected if her autonomy and agency are disregarded. Additionally, there may be legal or ethical implications depending on the specific circumstances. Overall, while helping others is generally considered ethical, it is important to approach each situation with sensitivity and respect for the individual's autonomy.
[{'Q': 'What is the definition of ethics?', 'A': "The definition of ethics refers to a set of moral principles or values that guide individuals or groups in determining what is right or wrong, good or bad, a

In [16]:
response

'Yes, it is ethical to say'

In [ ]:
def load_questions_from_json(json_path: str) -> list[str]:
    with open(json_path, 'r') as file:
        data = json.load(file)
        # Assume the questions are stored in a list under the key "questions"
        return data['questions']

async def answer_by_amplification(json_path: str):
    questions = load_questions_from_json(json_path)
    answers = []
    for question in questions:
        answer = await answer(question)
        answers.append((question, answer))
    return answers